# Task: add importance sampling capability

## 1. The original 2-qubit biased erasure noise model at erasure error rate of $p_e$ is injected as such

with probability $p_e/2 * (1- p_e)$, inject these four events:

    II herald 1, ZI herald 1, II herald 2, IZ herald 2,

with probability $(p_e/2)^2$, inject these four events:

    II double herald, IZ double herald, ZI double herald, ZZ double herald

which means there's on average $4*p_e/2 * (1- p_e) + 8*(p_e/2)^2 = 2p_e$ erasure after a 2-qubit gate


## 2. Sample_decode_job need to have attribute of num_Erasure and num_DEPOLARIZE2


### 1. how do erasure error mechanism behave when it's set to be erroneous?

Note that, it is WRONG to simply set $p_e = 1$ and inject these four events with probability $1/4$:

    II double herald, IZ double herald, ZI double herald, ZZ double herald

because most erasures exist in the form of single erasure, not double erasure

Therefore, every single erasure error mechanism should take in two bit of information during importance sampling


### 2. how do erasure error mechanism behave when it's set to be erroneous?

the original DEPOLARIZE2 introduces $24/15p_p = 8/5p_p$ Pauli errors, and if we view Y error as one X and one Z, that's $32/15p_p$ edges to be flipped in the matching graph.

But this detail is irrelevant. if we roll a dice we selected a DEPOLARIZE2 to be triggered, we just set $p_p$ = 1.

### 3. how to compute the weight of the logical error rate associated with samples of (num_Erasure = e, num_DEPOLARIZE2 = p)?

$A_{e,p}(p_e, p_p) = \binom{n_e}{e} p_e^e (1 - p_e)^{n_e - e} \times \binom{n_p}{p} p_p^t (1 - p_p)^{n_p - p}$

where $n_e$, $n_p$ is the total number of potential erasure/DEPOLARIZE2 error locations. 

For the circuit concerned, we have d noisy rounds, so there's $4*((d-1)^2+d)$ 2-qubit gates per round, and a total of $4d*((d-1)^2+d)$ 2-qubit gates



### 4. How to actually do the sampling? 

I need some sort of np.array to encode those potential error locations and then randomlly choose a certain number of them to flip

first, I need to create that numpy array


In [1]:
from surface_erasure_decoding import *

In [2]:
d = 3
after_cz_error_model = get_2q_error_model(p_e=0.01,
                                            p_z_shift=0, 
                                            p_p=0.001)
builder = easure_circ_builder(rounds = d,
                                distance= d,
                                after_cz_error_model=after_cz_error_model,
                                measurement_error=0
                                )
builder.generate_circuit_and_decoding_info()